<a href="https://colab.research.google.com/github/Anello92/AnaliseEstatistica/blob/master/projeto_previsao_vendas_rossmann_var_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.0 Imports

In [ ]:
!pip install inflection

In [ ]:
import pandas as pd
import numpy as np
import inflection
import math
import matplotlib.pyplot as plt
import seaborn as sns

### 0.1. Helper Functions

### 0.2 Loading Data

In [ ]:
df_sales_raw = pd.read_csv('train.csv', low_memory=False)
df_store_raw = pd.read_csv('store.csv', low_memory=False)


Neste segmento, estaremos utilizando a função read_csv do pandas, que é uma poderosa biblioteca de análise e manipulação de dados em Python. A função read_csv é usada para ler dados tabulares, como um arquivo CSV, e criar um DataFrame do pandas.

O primeiro argumento para a função read_csv é o caminho para o arquivo que queremos ler. No nosso caso, o arquivo está localizado na pasta 'data' e se chama 'Train.csv'.

O segundo argumento é low_memory. Neste exemplo, estamos definindo low_memory como False, o que instrui a função read_csv a carregar todo o arquivo na memória de uma vez, em vez de ler o arquivo em pedaços. Se low_memory fosse definido como True, a função leria o arquivo em pedaços para economizar memória.

A decisão de definir low_memory como True ou False depende da capacidade de memória do seu computador. Se você tentar carregar um arquivo muito grande e seu computador tiver memória limitada, pode receber um erro ou aviso.


In [ ]:
# Merge
df_raw= pd.merge(df_sales_raw, df_store_raw, how = 'left', on = 'Store')
df_raw.sample()


Carregados os arquivos, realizamos uma operação de 'merge' (fusão) em dois conjuntos de dados utilizando a função merge do pandas. Esta operação é semelhante a um 'JOIN' em SQL, onde os dados de dois (ou mais) DataFrames são combinados com base em uma coluna (ou várias colunas) em comum.

 Para fazer isso, utilizamos a função merge do pandas, que aceita vários argumentos.

O primeiro argumento é o DataFrame que servirá como referência para a operação de 'merge'. O segundo argumento é o DataFrame que será anexado ao primeiro. O argumento how especifica o tipo de 'merge' a ser realizado. No nosso caso, o valor é 'outer', o que significa que queremos um 'merge' que inclua todas as linhas de ambos os DataFrames, independentemente de haver uma correspondência entre as colunas.

O argumento on especifica a coluna (ou colunas) que serão usadas como chave para o 'merge'. No nosso caso, a coluna é 'Store', que está presente em ambos os DataFrames.

O resultado da operação de 'merge' é armazenado em uma nova variável chamada 'df'. Em resumo, a função merge é um método da classe pandas usado para combinar dois ou mais DataFrames com base em colunas em comum.

# 1.0. Descrição dos Dados

In [ ]:
df1 = df_raw.copy()

Uma sugestão é fazer cópias do DataFrame sempre que se muda de sessão de análise em um notebook. A prática é sugerida como uma medida preventiva contra a perda de dados originais durante a manipulação dos DataFrames em sessões subsequentes.

Esse procedimento pode evitar a necessidade de reexecutar o notebook do início, economizando tempo, especialmente quando se trabalha com grandes volumes de dados.

## 1.1 Rename Columns

In [ ]:
df_raw.columns

 É recomendável renomear as colunas para nomes mais intuitivos e fáceis de lembrar. Isso pode ajudar a acelerar o desenvolvimento posterior, já que os nomes das colunas serão frequentemente usados para explorar os dados, aplicar algoritmos, fazer gráficos, entre outros.

 Embora os nomes das colunas no exemplo atual sejam bastante organizados e estejam no formato 'camel case' (alternando entre letras maiúsculas e minúsculas), isso pode não ser o caso em um ambiente real, onde os nomes das colunas podem ser muito menos intuitivos.

 Portanto, é uma boa prática revisar e, se necessário, renomear as colunas no início do processo de análise de dados.

In [ ]:
cols_old = [
'Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
'CompetitionDistance', 'CompetitionOpenSinceMonth',
'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
'Promo2SinceYear', 'PromoInterval']

Prosseguindo com a análise de dados, é uma boa prática cria uma coluna chamada 'cols_old' no DataFrame. Esta coluna será preenchida com os nomes originais das colunas do DataFrame.

Uma vez que esta lista de nomes de colunas originais foi criada, o ideal é realizar a alteração dos nomes das colunas para um formato chamado 'snake case', que consiste em todas as palavras em minúsculas separadas por sublinhados.


In [ ]:
snakecase = lambda x: inflection.underscore(x)
cols_new = list(map(snakecase, cols_old))

Criamos a função snake_case. Para aplicá-la, utilizaremos outra função chamada map, que nos permite aplicar/mapear a função snake_case a todos os elementos de uma lista, nesse caso, a lista de nomes de colunas antigos em cols_old. O resultado dessa operação será armazenado em uma nova lista chamada 'cols_new'.

In [ ]:
df1.columns = cols_new
df1.columns

## 1.2 Data Dimensions

Um passo crucial na descrição de nossos dados é determinar as dimensões do nosso DataFrame - o número de linhas e colunas. Para isso, utilizamos o método shape que fornece essas informações.

In [ ]:
print( 'Number of Rows: {}'.format(df1.shape[0]))
print( 'Number of Cols: {}'.format(df1.shape[1]))

Ao executar essas instruções, descobrimos que nosso DataFrame possui 1.017.209 linhas distribuídas em 18 colunas. A quantidade de dados é considerável, mas gerenciável para os nossos recursos computacionais atuais. No entanto, se for necessário lidar com conjuntos de dados maiores, existem opções como servidores AWS, Google Cloud ou Kaggle, que oferecem recursos computacionais robustos gratuitamente ou com custos associados.

## 1.3 Data Types

 O próximo passo na descrição dos dados é examinar os tipos de dados presentes no DataFrame. Para isso, utilizamos o método **dtypes**, que nos permite ver a coluna e o tipo de dado correspondente sem a necessidade de usar parênteses.

In [ ]:
df1.dtypes

 Isso nos permite identificar que colunas como 'Store' são do tipo int64 e 'Date' é do tipo objeto.

No caso da coluna 'Date', queremos alterar o formato de dados para que possamos trabalhar com ele como uma data.

In [ ]:
df1['date'] = pd.to_datetime(df1['date'])
df1.dtypes

Para fazer isso, utilizamos o método to_datetime no pandas e aplicamos diretamente na coluna 'Date' para convertê-la para o tipo de data correto. Quando verificamos novamente o tipo de dados na coluna 'Date', confirmamos que agora é datetime64, que é o formato adequado para trabalhar com datas no pandas.

### Check NA

O próximo passo na análise é a realização de uma verificação de dados nulos (NaN), que é essencial para garantir a qualidade dos dados que estamos trabalhando.

Começamos usando o método isna() em nosso DataFrame, que nos permite ver todas as linhas que contêm pelo menos uma coluna com valor ausente ou nulo. No entanto, em vez de visualizar essas linhas individualmente, queremos a soma dessas linhas por coluna, o que é feito utilizando o método sum().

In [ ]:
df1.isna().sum()

Ao executar esses comandos, podemos ver que algumas de nossas colunas não contêm valores ausentes, enquanto outras possuem alguns. Precisamos tratar esses valores ausentes e, geralmente, existem três métodos para lidar com eles:

- Descartar as linhas: Esse método envolve simplesmente descartar qualquer linha que contenha pelo menos um valor ausente. A principal vantagem é que é rápido e fácil, mas a desvantagem significativa é que você pode estar descartando informações úteis para o algoritmo aprender padrões, o que pode prejudicar a performance do seu modelo se você não tiver muitos dados.

- Usar algoritmos de aprendizado de máquina: Há algoritmos e métodos que podem preencher valores ausentes com base no comportamento geral da coluna. Por exemplo, você pode preencher os valores ausentes com a média, mediana ou moda da coluna, ou até mesmo usar algoritmos mais avançados que estimam o que o valor poderia ser se não estivesse ausente.

- Usar conhecimento de negócio: Às vezes, os valores ausentes estão lá devido a alguma lógica de negócio definida inicialmente. Se soubermos essa regra, podemos preencher os valores ausentes de forma adequada e recuperar os dados.

## 1.5 Fillout NA
Vamos iniciar uma nova subseção, denominada 'Fill-na', com o objetivo de preencher os dados ausentes.


Começaremos lidando com a coluna 'competition_distance'. Retornando à descrição da coluna, sabemos que essa coluna indica a distância, em metros, do concorrente mais próximo.

In [ ]:
# competition_distance
# competition_open_since_month
# competition_open_since_year
# promo2_since_week
# promo2_since_year
# promo_interval

### Existem três maneiras principais de lidar com dados ausentes (NaN):

A primeira maneira é excluir todos os dados ausentes. A principal vantagem desse método é a rapidez, mas a desvantagem é a perda substancial de informações.

A segunda maneira é usar algoritmos de aprendizado de máquina para substituir dados ausentes com base no comportamento da coluna. Por exemplo, podemos calcular a mediana ou a média e substituir todos os dados ausentes por esses valores. Algoritmos mais sofisticados podem fazer agrupamentos ou até mesmo prever esses valores ausentes.

A terceira maneira, e a que vamos adotar neste tutorial, é pensar do ponto de vista do negócio. Embora não sejamos especialistas, é útil fazer esse tipo de exercício. Pensar sobre o motivo pelo qual esses dados estão ausentes pode oferecer insights valiosos.



### 1. Competition Distance
Vamos considerar a coluna 'competition_distance'. Uma maneira de interpretar a ausência de dados aqui é que a distância até o concorrente mais próximo é tão grande que é efetivamente o mesmo que não haver concorrente próximo.

In [ ]:
df1['competition_distance'].max()

Com a ausência de valores em 'competition_distance', iremos substituí-los por um número extremamente grande, que chamaremos de 'max_value'. Para definir este 'max_value', vamos primeiro procurar o valor máximo existente na coluna 'competition_distance'. No nosso caso, encontramos o valor de 75.860 metros como o mais distante concorrente.

In [ ]:
df1['competition_distance'] = df1['competition_distance'].apply(
    lambda x: 200000.0 if math.isnan(x) else x)

Vamos escolher 'max_value' como 200.000 metros, um valor bem maior que a distância máxima existente na coluna 'competition_distance'. Valores ausentes serão substituídos por 'max_value', indicando que não há concorrente próximo nesses casos.

Se um valor estiver presente, será retornado como está. Aplicaremos essa lógica ao nosso conjunto de dados usando a função lambda, que será aplicada apenas na coluna 'competition_distance'. O resultado substituirá a coluna original.

Após aplicar a função e recalcular a soma de linhas NaN, vemos que não há mais valores ausentes na coluna 'competition_distance', mostrando que lidamos com os dados ausentes com sucesso.

In [ ]:
df1.isna().sum()

Agora, a coluna 'competition_distance' não possui mais valores ausentes, e o valor máximo é 200.000 metros, como definido anteriormente.

In [ ]:
df1['competition_distance'].max()

### 2. Competition Open Since Month
Avançando, vamos analisar a coluna 'competition_open_since'. Esta coluna indica o mês e o ano aproximado em que o concorrente mais próximo foi inaugurado.

Podemos supor que os valores ausentes nesta coluna podem ser por duas razões: primeiro, a loja pode não ter um concorrente próximo, logo não existe uma data de abertura para tal concorrente. Segundo, a loja pode ter um concorrente próximo, mas desconhecemos a data de abertura deste, seja porque o concorrente abriu antes da loja ou porque abriu posteriormente.

Para substituir os valores ausentes nesta coluna, copiaremos a data de venda correspondente na linha em questão para a coluna 'competition_open_since'.

In [ ]:
df1.sample()

Agora vamos considerar a coluna 'competition_open_since' em uma linha de exemplo onde uma venda foi feita. Por exemplo, se a linha representar uma venda feita pela loja 283 em 20/05/2014, onde vendeu 7300 itens, e os dados mostram um valor ausente para 'competition_open_since', substituiremos esse valor ausente com a data da venda.

Vamos extrair apenas o mês da data da venda para preencher os valores ausentes na coluna 'competition_open_since_month'. Isso se deve à nossa previsão de que variáveis temporais podem ser cruciais na modelagem do comportamento dos dados.

A lógica é que uma loja sem concorrência tem um nível estável de vendas, que provavelmente cairá quando um concorrente abrir. Com o tempo, as vendas podem se recuperar à medida que os clientes se acostumam com ambas as lojas. Assim, o tempo desde a abertura de um concorrente pode influenciar as flutuações de vendas.

Reconhecemos uma incongruência em usar a data da venda como a data de abertura do concorrente, especialmente se supomos que o concorrente ainda não abriu. No entanto, prosseguiremos com essa abordagem para avaliar seu impacto no modelo. Se necessário, podemos ajustá-la em futuras iterações.

A forma como implementaremos essa substituição será semelhante ao que fizemos para 'competition_distance'.






In [ ]:
df1['competition_open_since_month'] = df1.apply(lambda x: x['date'].month
                                                if np.isnan(x['competition_open_since_month'])
                                                else x['competition_open_since_month'], axis=1)

Portanto, se a coluna 'competition_open_since_month' estiver vazia, iremos preencher com o mês da coluna 'date'.

Se essa condição não for verdadeira, iremos simplesmente retornar o valor original, pois já temos o mês em que a competição foi aberta.

Para aplicar essa lógica, vamos usar a função lambda novamente. Dentro desta função, podemos substituir 'df1' por 'x', já que tudo dentro da função é referido como 'x'.

Para aplicar isso aos nossos dados, vamos usar a função 'apply' e aplicá-la ao longo das colunas. Fazemos isso porque estamos trabalhando com mais de uma coluna - 'competition_open_since_month' e 'date'. Quando trabalhamos com mais de uma coluna, precisamos explicitamente aplicar a função ao longo das colunas.

Finalmente, o resultado dessa função será usado para substituir a coluna original 'competition_open_since_month'.






### 3. Competition Open Since Year
Vamos substituir todas as ocorrências de 'month' por 'year' na linha atual.


In [ ]:
df1['competition_open_since_year'] = df1.apply(lambda x: x['date'].year
                                               if np.isnan(x['competition_open_since_year'])
                                               else x['competition_open_since_year'], axis=1)

### 4. Promo 2 Since Week
A coluna 'promo2' indica se uma loja está participando de uma continuação de uma promoção. Se o valor é zero, a loja não está participando; se é um, a loja está participando. Se os dados para 'promo2' são NaN, isso indica que a loja optou por não participar dessa promoção contínua, logo não há uma semana de início para essa promoção.

Para lidar com esses valores ausentes, usaremos uma abordagem semelhante à que usamos para a coluna 'competition_distance': substituir os NaNs pelo valor da data naquela linha. Quando calculamos a distância em tempo, isso permitirá ao algoritmo reconhecer que temos essa promoção ativa desde uma determinada semana.

Para implementar essa substituição, copiamos e colamos o código que usamos para 'competition_distance', fazendo algumas pequenas alterações. Em vez de 'competition', procuramos 'promo2', e em vez de extrair o mês da data, queremos extrair a semana.

In [ ]:
df1['promo2_since_week'] = df1.apply(lambda x: x['date'].week
                                     if np.isnan(x['promo2_since_week'])
                                     else x['promo2_since_week'], axis=1)

### 5. Promo 2 Since Year
Para implementar essa substituição, copiamos e colamos o código que usamos para 'promo2_since_week', fazendo algumas pequenas alterações para year e extrair o ano.

In [ ]:
df1['promo2_since_year'] = df1.apply(lambda x: x['date'].year
                                     if np.isnan(x['promo2_since_year'])
                                     else x['promo2_since_year'], axis=1)

Este código faz o seguinte:

- Usa a função apply para aplicar uma função lambda a cada linha do dataframe (df1.apply(lambda x: ..., axis=1)).
- Na função lambda, verificamos se 'promo2_since_year' é NaN para essa linha (if math.isnan(x['promo2_since_year'])).
- Se for NaN, substituímos pelo número da semana da data dessa linha (x['date'].year).
- Se não for NaN, mantemos o valor original (else x['promo2_since_year']).

### 6. Promo Interval
Promo Interval" descreve os meses em que uma promoção chamada "Promo 2" foi iniciada. Por exemplo, se os valores forem "Fevereiro, Maio, Agosto, Novembro", isso indica que a promoção foi realizada nesses meses específicos.

In [ ]:
month_map = {1: 'Jan', 2: 'Fev', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dez'}

df1['promo_interval'].fillna(0, inplace = True)

df1['month_map'] = df1['date'].dt.month.map(month_map)

df1['is_promo'] = df1[['promo_interval', 'month_map']].apply(lambda x: 0
                                                             if x['promo_interval'] == 0
                                                             else 1 if x['month_map'] in x['promo_interval'].split(',') else 0, axis = 1)

In [ ]:
df1.sample(5).T

A primeira etapa na manipulação desses dados é fazer um "split" nesta coluna para criar uma lista dos meses da promoção. Em seguida, uma nova coluna chamada "month_map" será criada com base na data da promoção. Se a data da promoção estiver na lista de meses, um valor indicando que a "Promo 2" estava ativa será inserido nessa nova coluna.

Depois de criar a coluna auxiliar, os meses são convertidos de números para suas representações de string correspondentes. Por exemplo, janeiro seria representado como "1". Uma função é então aplicada para fazer a substituição direta na coluna.

Em seguida, os valores da coluna "Promo Interval" são substituídos por seus equivalentes numéricos. Para fazer isso, um dicionário é criado e aplicado à coluna. Isso cria uma coluna chamada "month_map" que contém o mês em que a promoção ocorreu.

Uma função é então aplicada para comparar se o mês de uma promoção está na lista de meses de "Promo 2". Se o mês estiver na lista, é retornado um valor de "1". Se o mês não estiver na lista, é retornado um valor de "0". Além disso, se uma loja não participou da "Promo 2" (indicado por um valor de "0" na coluna "Promo Interval"), um valor de "0" também é retornado.


In [ ]:
df1.isna().sum()

## 1.6 Change Types
Quando operações são realizadas em colunas de um conjunto de dados, é vital verificar novamente os tipos de dados dessas colunas. A razão para isso é que as operações podem inadvertidamente alterar os tipos de dados originais.

Pode haver situações em que é necessário mudar o tipo de dados de uma coluna para um diferente. Por exemplo, se quisermos converter o tipo de dados 'float' da coluna 'competition' para 'int' (um número inteiro), podemos usar o método astype().


In [ ]:
df1['competition_open_since_month'] = df1['competition_open_since_month'].astype(int)
df1['competition_open_since_year'] = df1['competition_open_since_year'].astype(int)

df1['promo2_since_week'] = df1['promo2_since_week'].astype(int)
df1['promo2_since_year'] = df1['promo2_since_year'].astype(int)

## 1.7 Descriptive Statistics
A estatística descritiva é uma ferramenta essencial para a análise de dados, sendo particularmente útil em duas áreas: obtenção de conhecimento de negócios e detecção de erros nos dados.

Em primeiro lugar, a estatística descritiva proporciona uma visão geral dos dados, permitindo obter insights sobre os negócios. Isto é feito através de métricas de estatísticas descritivas, que se dividem em duas categorias principais: **métricas de tendência central** e **métricas de dispersão**.

As métricas de tendência central, como a média e a mediana, fornecem um resumo dos dados em um único valor. Em essência, elas permitem representar uma variedade de dados através de uma média ou mediana. No entanto, embora essas métricas possam fornecer uma visão geral, elas não revelam como os dados estão distribuídos em torno dessa referência central. É aqui que entram as métricas de dispersão.

As métricas de dispersão, que incluem variância, desvio padrão e valores mínimos e máximos, fornecem informações sobre a distribuição dos dados. Elas indicam se os dados estão concentrados próximo à média ou dispersos em torno dela. Além disso, duas outras medidas de dispersão, chamadas "skewness" (assimetria) e "kurtosis" (curtose), podem ser usadas para entender melhor a distribuição dos dados. A skewness refere-se ao grau de distorção da distribuição em relação a uma distribuição normal, enquanto a curtose descreve a concentração dos dados, indicando se os dados têm um "pico" alto (curtose positiva) ou são mais dispersos (curtose negativa).

Ao trabalhar com a estatística descritiva, é importante separar as variáveis numéricas das categóricas, pois a forma como cada tipo é tratado pode variar. Para selecionar variáveis numéricas, você pode utilizar um método de seleção em sua ferramenta de análise de dados, como select_dtypes() no pandas, uma biblioteca do Python, e passar uma lista dos tipos de dados que você deseja selecionar.

In [ ]:
num_attributes = df1.select_dtypes(include = ['int64', 'float64'])
cat_attributes = df1.select_dtypes(exclude = ['int64', 'float64', 'datetime64[ns]'])

Neste exemplo, 'df1' é o DataFrame original e 'num_attributes' é o novo DataFrame contendo apenas as colunas numéricas. Podemos visualizar algumas linhas aleatórias do DataFrame usando o método sample(), como mostrado abaixo

In [ ]:
num_attributes.sample(2)

Agora, 'cat_attributes' contém todas as colunas categóricas do DataFrame original. Como antes, podemos visualizar algumas linhas aleatórias do DataFrame usando o método sample().

In [ ]:
cat_attributes.sample(2)

Após separar os dados numéricos e categóricos, podemos proceder para calcular diferentes métricas relevantes para cada tipo de dado. Por exemplo, para dados numéricos, podemos calcular a média, mediana, desvio padrão, etc. Para dados categóricos, podemos contar a frequência de diferentes categorias, encontrar a categoria mais comum, etc.

### 1.7.1 Numerical Attributes
Vamos abordar o cálculo de métricas de tendência central para variáveis numéricas. As métricas de tendência central resumem o conjunto de dados em um único valor representativo, fornecendo um 'centro' em torno do qual os dados estão distribuídos. As métricas de tendência central mais comuns são a média e a mediana.

In [ ]:
# Central Tendency - mean, median
ct1 = pd.DataFrame(num_attributes.apply(np.mean)).T
ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

# Dispersion - std, min, max, skew, kurtosis
d1 = pd.DataFrame(num_attributes.apply(np.std)).T
d2 = pd.DataFrame(num_attributes.apply(min)).T
d3 = pd.DataFrame(num_attributes.apply(max)).T
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T

Depois de todas essas métricas calculadas, podemos concatená-las em um único DataFrame para facilitar a visualização. Usamos o método concat() da biblioteca pandas para isso:

In [ ]:
# Concatenate
m = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
m

Neste ponto, 'm' é um DataFrame que contém todas as métricas calculadas para cada coluna numérica. Cada linha representa uma métrica, e cada coluna representa uma coluna numérica do DataFrame original.

Considere uma coluna 'Sales' em um conjunto de dados. Com base nas estatísticas, o valor mínimo é zero, indicando que houve dias em que nenhuma venda foi realizada, talvez devido ao fechamento de lojas. O valor máximo é de 41.000, indicando o maior volume de vendas em um dia. O 'range', ou a diferença entre o máximo e o mínimo, é, portanto, 41.000.

Quando a média e a mediana de um conjunto de dados são semelhantes, como neste caso, **indica que a distribuição é próxima de uma distribuição normal**, sem um deslocamento significativo para a esquerda ou para a direita. Além disso, o skewness (medida da assimetria da distribuição) e kurtosis (medida da "cauda" ou outliers da distribuição) também estão próximos de zero, corroborando a suposição da normalidade.


### 1.7.2 Categorical Attributes
Para realizar uma análise descritiva dos dados categóricos, é útil usar gráficos para visualizar as informações. Um tipo de gráfico muito útil para esse propósito é o gráfico de boxplot. Este gráfico permite visualizar diversas medidas estatísticas, como medidas de tendência central e dispersão, em um único lugar, facilitando a comparação entre as categorias.

Entretanto, antes de construir o boxplot, pode ser útil verificar a quantidade de níveis ou valores únicos que cada variável categórica possui. Isso pode ser feito com o uso da função nunique do pandas em Python, que retorna o número de elementos distintos em cada coluna de um dataframe. A função nunique pode ser aplicada a todas as colunas de um dataframe utilizando o método apply.



In [ ]:
cat_attributes.apply(lambda x: x.unique().shape[0])

No exemplo fornecido, as variáveis categóricas analisadas incluem 'state_holiday', 'store_type', 'assortment, 'promot_interval' e 'month_map'. Cada uma dessas variáveis possui três níveis distintos.

In [ ]:
sns.boxplot(x = 'state_holiday', y = 'sales', data = df1)

Estamos analisando a dispersão de 'sales' em cada nível de uma variável categórica. Para isso, podemos definir 'state_holiday' como o eixo x (a categoria) e 'sales' como o eixo y (o valor a ser medido). Os dados serão extraídos do nosso DataFrame, digamos df1, que contém os dados categóricos.

No entanto, pode ser que, ao visualizar o boxplot, os dados estejam distorcidos ou difíceis de interpretar. Uma razão comum para isso é a presença de valores que não são úteis para a análise, como vendas iguais a zero correspondendo a dias em que as lojas estão fechadas.

Para resolver esse problema, podemos filtrar os dados antes de plotar o gráfico. Por exemplo, podemos querer visualizar apenas as vendas que são maiores que zero e que ocorreram em dias que não são feriados. Para fazer isso, podemos utilizar a operação de indexação booleana do pandas para criar um novo DataFrame que satisfaz essas condições:


In [ ]:
aux1 = df1[(df1['state_holiday'] != '0') & (df1['sales'] > 0)]

plt.figure(figsize=(15, 6))  # Aqui, 15 representa a largura e 6 a altura.

plt.subplot(1, 3, 1)
sns.boxplot(x = 'state_holiday', y = 'sales', data = aux1)

plt.subplot(1, 3, 2)
sns.boxplot(x = 'store_type', y = 'sales', data = aux1)

plt.subplot(1, 3, 3)
sns.boxplot(x = 'assortment', y = 'sales', data = aux1)

plt.tight_layout()
plt.show()

Acima utilizamos a biblioteca Seaborn para criar boxplots de diferentes variáveis categóricas em relação às vendas. Por exemplo, analisamos as vendas de acordo com o tipo de feriado 'state_holiday' e o tipo de loja 'store_type'.

Para comparar visualmente todas as variáveis de uma só vez, usamos a função plt.subplot(), que nos permite traçar múltiplos gráficos lado a lado. No caso, configuramos para uma linha e três colunas, correspondendo aos três boxplots.

A mediana, indicada pela linha no meio da 'caixa' do boxplot, representa o valor do 50º percentil, ou seja, o ponto onde metade dos dados está abaixo e metade está acima. O primeiro quartil (Q1) é o 25º percentil, indicando onde 25% dos dados estão abaixo e 75% estão acima, enquanto o terceiro quartil (Q3) é o 75º percentil, marcando onde 75% dos dados estão abaixo e 25% estão acima.

Observamos que o 'state_holiday' do tipo 'B' tem uma mediana maior de vendas do que os outros tipos. Em relação aos tipos de loja em 'store_type', a loja do tipo 'B' tem mais outliers e uma mediana de vendas significativamente mais alta em comparação às outras.

Cada boxplot fornece uma rica informação sobre a distribuição das vendas para cada categoria, possibilitando identificar onde a maioria das vendas está concentrada (em torno da mediana) e como elas estão distribuídas entre o mínimo e o máximo.

Essa análise usando boxplots facilita a compreensão da relação entre diferentes variáveis categóricas e as vendas, sendo uma ferramenta poderosa para a análise exploratória de dados.

## Status Projeto Rossmann
Seguindo o nosso projeto Rossmann de previsão de vendas por loja para as próximas seis semanas em um contexto de negócio.

Inicialmente, identificou-se a causa raiz desse problema, que é a dificuldade do CEO em determinar a quantidade de investimento necessário para a reforma das lojas. Com base nessa questão, torna-se pertinente realizar uma previsão de vendas para auxiliar na tomada de decisões.

A próxima etapa é a coleta de dados, que, no caso deste projeto, envolve o download dos dados disponíveis na plataforma do Kaggle. Em situações reais, a coleta pode envolver várias fontes de dados relevantes para a criação da tabela final utilizada na análise.

Em seguida, é necessário realizar a etapa de limpeza dos dados, que consiste em garantir a correção e consistência dos tipos de dados, substituindo valores faltantes com base em critérios estabelecidos. Além disso, é importante realizar um resumo dos dados para obter uma compreensão inicial de sua dimensão e características.

No momento, estamos na etapa de exploração dos dados, que envolve três tarefas principais. A primeira é a **criação de variáveis derivadas**, ou seja, a derivação de novas variáveis com base nas existentes para **melhorar a representação dos dados** e capturar informações relevantes.

Em seguida, é necessário criar uma lista de hipóteses que serão testadas durante a exploração dos dados. Essas hipóteses podem estar relacionadas a padrões de comportamento, correlações entre variáveis ou quaisquer outras suposições relevantes para o problema em questão.

Por fim, é importante validar as hipóteses levantadas durante a exploração dos dados, utilizando técnicas estatísticas e visuais para confirmar ou refutar cada uma delas. Essa validação auxilia na compreensão do conjunto de dados e no direcionamento das próximas etapas do projeto.

Após a conclusão da etapa de exploração dos dados, restarão outras etapas a serem executadas para finalizar o primeiro ciclo da solução proposta. No entanto, com o progresso alcançado até o momento, será possível entregar a versão inicial desta solução, o que proporcionará insights para o processo de tomada de decisões no setor de varejo.

## Mindmap de Hipóteses

O mapa mental de hipóteses desempenha um papel crucial na etapa de análise exploratória de dados, fornecendo uma orientação clara e direcionada para alcançar insights valiosos de forma eficiente.

Esse mapa é essencialmente um roteiro que orienta quais análises devem ser realizadas para validar as hipóteses definidas. Além disso, o mapa indica quais **variáveis devem ser derivadas para a realização dessas análises**. Em outras palavras, o mapa mental fornece uma estrutura clara das análises necessárias e das variáveis envolvidas, permitindo direcionar a exploração dos dados de maneira eficiente.

Ao utilizar o mapa mental de hipóteses, teremos um guia que ajudará a determinar o nível de detalhamento necessário para cada ciclo do projeto. Isso significa que poderemos identificar até onde precisa aprofundar a análise durante a fase inicial. Com esse direcionamento claro, será possível conduzir a análise de maneira mais rápida e direta, trazendo insights valiosos para cada estágio do ciclo de desenvolvimento.

Antes de prosseguirmos para a criação do mapa mental de hipóteses, gostaria de destacar os três elementos que compõem esse mapa. Esses elementos são fundamentais para a compreensão e aplicação efetiva, garantindo que possamos aproveitar ao máximo essa ferramenta em nosso projeto de previsão de vendas Rossmann.







## Elementos do Mapa Mental de Hipóteses

O mapa mental é composto por três elementos-chave: o **fenômeno**, os **agentes** e os **atributos dos agentes**. Cada um desses elementos desempenha um papel fundamental na definição das hipóteses a serem exploradas e validadas durante a análise dos dados.

O primeiro elemento, o fenômeno, refere-se ao **que estamos tentando medir ou modelar**. No contexto do nosso projeto, esse fenômeno é a **previsão de vendas**. Podemos considerar outros exemplos, como detecção de objetos em uma imagem, classificação de imagens entre gato e cachorro ou clusterização de clientes para criação de personas. **O fenômeno é o aspecto central que desejamos compreender e modelar para ensinar os algoritmos de machine learning**.

O segundo elemento do mapa mental são os agentes, ou seja, as **entidades que impactam o fenômeno** de alguma forma. No caso das **vendas**, os agentes podem ser **clientes**, **lojas** e **produtos**. É importante reconhecer que esses agentes influenciam diretamente as vendas, podendo contribuir para o aumento ou diminuição das mesmas. Por exemplo, **um maior número de clientes tende a aumentar as vendas**, enquanto um aumento no **preço do produto pode resultar em vendas menores**. Portanto, é fundamental identificar e compreender todos os agentes relevantes nesse contexto.

O terceiro e último elemento são os atributos dos agentes. Cada agente pode ser descrito por uma série de atributos. Por exemplo, no caso dos clientes, podemos considerar atributos como **idade**, **escolaridade**, **estado civil**, **número de filhos**, **frequência de visitas à loja**, **salário**, **educação** e **profissão**. Esses atributos ajudam a descrever as características e peculiaridades de cada agente, fornecendo informações valiosas para análise.

O objetivo principal do mapa mental de hipóteses é derivar uma lista de hipóteses a serem testadas e validadas. Com base nessa lista, é possível priorizar as hipóteses e realizar a análise dos dados de forma direcionada.

Durante essa análise, é possível gerar insights relevantes para o projeto. Os insights podem ser gerados de duas maneiras: através de surpresas, quando uma informação desconhecida é descoberta, ou por meio da contraposição de crenças, quando uma hipótese é confrontada e os resultados obtidos desafiam as expectativas iniciais.







### Como escrever as Hipóteses?
O fenômeno que estamos tentando modelar é o Store Sales, ou seja, as vendas diárias das lojas da Rossmann. Esse é o aspecto central que desejamos entender e prever em nosso projeto. Além disso, existem diferentes agentes que impactam esse fenômeno.

Os clientes são um dos agentes envolvidos. Eles são as pessoas que realizam compras nas lojas. Além disso, temos as próprias lojas, que também são agentes relevantes. Cada loja pode ter diferentes características, como localização geográfica e tamanho.

Os produtos também são agentes que influenciam o fenômeno das vendas. Cada produto pode ter atributos específicos, como preço, quantidade em estoque, exposição na loja, participação em promoções e intensidade de marketing. Esses atributos dos produtos desempenham um papel importante na influência das vendas.

No contexto das lojas, existem atributos que descrevem esse agente específico. Esses atributos podem incluir tamanho da loja, sortimento de produtos disponíveis, quantidade de estoque, presença de concorrentes próximos e número de funcionários. Essas características ajudam a descrever e entender melhor o comportamento das lojas em relação às vendas.

Além disso, é importante considerar os elementos temporais relacionados ao fenômeno das vendas. Nesse sentido, podemos incluir atributos como ano, mês, dia, hora, semana do ano, fim de semana, feriados e promoções específicas do mercado, como saldões. Esses atributos fornecem informações sobre a sazonalidade e os padrões temporais que podem influenciar as vendas ao longo do tempo.

Outro aspecto relevante é a localização das lojas. Cada loja pode ter atributos de localização, como proximidade de escolas, hospitais ou outros pontos de interesse. Esses atributos ajudam a compreender como a localização geográfica das lojas pode influenciar as vendas.

Agora que temos todos esses elementos mapeados, podemos transformar o mapa mental de hipóteses em uma lista de hipóteses a serem testadas e validadas. Cada ramo do mapa mental nos fornece uma hipótese que pode ser julgada e priorizada. Durante a análise dos dados, exploraremos essas hipóteses para confirmá-las ou refutá-las.

Por exemplo, considerando os clientes como agentes, podemos levantar a hipótese de que clientes com alto volume de compra estão relacionados a vendas maiores. Da mesma forma, podemos supor que clientes com maior número de filhos também contribuem para vendas mais altas, pois há mais demanda por produtos. Por outro lado, clientes com altos salários podem não fazer compras na loja, impactando negativamente as vendas. Essas são apenas algumas hipóteses que podemos derivar do mapa mental.

Agora surge a pergunta: como criamos esse mapa mental de hipóteses? Em projetos particulares, podemos desenvolvê-lo com base em nosso conhecimento prévio sobre o fenômeno. Podemos realizar pesquisas, ler materiais relevantes e criar uma estrutura em nossa mente.

Já em uma empresa, como cientistas de dados, o mapa mental de hipóteses geralmente surge de reuniões de brainstorming. Nesse momento, convidamos pessoas de diferentes áreas, como engenharia, produtos e logística, e perguntamos a eles quais alavancas ou ações eles acreditam que impactam as vendas, seja aumentando-as ou diminuindo-as.

Cada participante contribui com suas percepções e experiências. Por exemplo, alguém do marketing pode mencionar que investir mais em anúncios ou aumentar o orçamento resulta em aumento de vendas. Já a pessoa responsável pelos produtos pode compartilhar que uma variedade diversificada de produtos e a utilização de diferentes cores aumentaram as vendas no passado. A área de logística pode enfatizar que uma entrega rápida e uma experiência positiva do cliente são fundamentais para impulsionar as vendas.

A partir dessas discussões e anotações, criamos a estrutura do mapa mental de hipóteses, mapeando os agentes envolvidos no fenômeno e seus atributos.

O objetivo final desse mapa é derivar uma lista de hipóteses que serão testadas e validadas. Durante a análise dos dados, exploraremos essas hipóteses e geraremos insights relevantes. Esses insights podem surgir por meio de surpresas, quando descobrimos informações desconhecidas anteriormente, ou por meio da contraposição de crenças, quando os resultados obtidos desafiam as expectativas iniciais.


# 2.0 Feature Engineering

Ao escrever as hipóteses, é importante entender que elas são basicamente apostas ou suposições sobre algo em relação ao fenômeno que estamos modelando. Por exemplo, uma hipótese pode ser: "Lojas de maior porte deveriam vender mais". Vamos analisar essa sentença de aposta. Primeiramente, é fundamental reconhecer que essa é apenas uma aposta, não sabemos se é verdadeira ou não. Precisamos validar essa hipótese com base nos dados disponíveis.

Observando a hipótese, podemos destacar os elementos importantes. No exemplo citado, o atributo é o tamanho da loja. "Vender" é a variável resposta, ou seja, o fenômeno que estamos tentando modelar, que são as vendas das lojas. As palavras "Maior" e "Mais" representam a aposta em si. Neste caso, a aposta é que lojas de maior porte deveriam vender mais.

Uma abordagem interessante é relacionar cada atributo do conjunto de dados com a variável resposta, fazendo uma aposta com base em nossa intuição. Por exemplo, acredita-se que se um determinado atributo aumenta, a variável resposta também aumenta, ou se o atributo diminui, a variável resposta também diminui. Essas hipóteses guiarão nossa investigação, permitindo que façamos apostas em relação aos atributos e sua relação com a variável resposta.

É importante mencionar que essa relação entre atributos e variável resposta não é de causa e efeito. Por exemplo, não podemos afirmar categoricamente que lojas de maior porte vendem mais simplesmente porque aumentamos o tamanho da loja. As vendas podem aumentar devido a outros fatores, como um maior número de clientes, que por sua vez é impulsionado pelo tamanho da loja. Portanto, devemos entender que essas hipóteses representam uma correlação entre os atributos e a variável resposta.

A seguir, apresentaremos mais exemplos de hipóteses derivadas do mapa mental. Por exemplo:
- "Lojas com maior sortimento deveriam vender mais" - "Sortimento" é um atributo do agente loja, "vender" é a variável resposta e "maior" é a aposta.
- "Lojas com mais competidores próximos deveriam vender menos" - "Competidores" é um atributo das lojas, "vender" é a variável resposta e "mais" e "menos" são as apostas.

## 2.1 Mapa Mental de Hipóteses

## 2.1.1. Hipóteses Loja
Na subseção 2.1.1, que trata das hipóteses relacionadas às lojas, iremos criar nossas hipóteses. Podemos utilizar um formato marcado para facilitar a visualização. A primeira hipótese será sobre o número de funcionários nas lojas. Baseando-se no mapa mental, podemos criar as hipóteses:
1. Lojas com maior quadro de funcionários deveriam vender mais.
2. Lojas com maior capacidade de estoque deveriam vender mais.
3. Lojas com maior porte deveriam vender mais.
4. Lojas com menor porte deveriam vender menos.
5. Lojas com maior sortimento deveriam vender mais.
6. Lojas com competidores mais próximos deveriam vender menos.
7. Lojas com competidores há mais tempo deveriam vender mais.

É importante destacar que essas hipóteses serão validadas durante a etapa de análise exploratória de dados. Cada uma delas será analisada com base nos dados disponíveis, permitindo confirmar ou refutar sua influência nas vendas.

## 2.1.2. Hipóteses Produto

1. Lojas que investem mais em Marketing deveriam vender mais.
2. Lojas com mais produtos expostos nas vitrines deveriam vender mais.
3. Lojas com produtos com preços menores deveriam vender mais.
4. Lojas que têm preço menores por mais tempo nos produtos deveriam vender mais.
5. Lojas com promoções mais agressivas deveriam vender mais.
6. Lojas com promoções ativas por mais tempo deveriam vender mais.
7. Lojas com mais dias de promoção deveriam vender mais.
8. Lojas com mais promoções consecutivas deveriam vender mais.

## 2.1.3. Hipóteses Tempo
1. Lojas abertas durante o feriado de Natal deveriam vender mais.
2. Lojas deveriam vender mais ao longo dos anos.
3. Lojas deveriam vender mais no segundo semestre do ano.
4. Lojas deveriam vender mais depois dos dia 10 de cada mês.
5. Lojas deveriam vender menos aos finais de semana.
6. Lojas deverim vender menos durante os feriados escolares.

## 2.2. Lista Final de Hipóteses
Uma etapa importante é a **priorização das hipóteses** que iremos utilizar durante a análise de dados. Para isso, utilizamos um critério simples e eficaz: a **disponibilidade dos dados necessários no momento**.

Algumas hipóteses podem ser comprovadas ou refutadas utilizando os dados já disponíveis, enquanto outras exigem tempo para coletar, organizar e preparar os dados para análise. Portanto, **priorizamos as hipóteses que podem ser validadas de imediato**, caso sejam relevantes para o modelo em questão:

1. Lojas com maior sortimento deveriam vender mais.
2. Lojas com competidores mais próximos deveriam vender menos.
3. Lojas com competidores há mais tempo deveriam vender mais.
4. Lojas com promoções ativas por mais tempo deveriam vender mais.
5. Lojas com mais dias de promoção deveriam vender mais.  
6. Lojas com mais promoções consecutivas deveriam vender mais.
7. Lojas abertas durante o feriado de Natal deveriam vender mais.
8. Lojas deveriam vender mais ao longo dos anos.
9. Lojas deveriam vender mais no segundo semestre do ano.
10. Lojas deveriam vender mais depois do dia 10 de cada mês.
11. Lojas deveriam vender menos aos fins de semana.
13. Lojas deveriam vender menos durante os feriados escolares.

Dentre as hipóteses relacionadas às **lojas**, podemos considerar a hipótese de que lojas com um maior número de funcionários tendem a ter um maior volume de vendas. No entanto, essa hipótese requer dados específicos sobre o número de funcionários, os quais não temos disponíveis no momento. Da mesma forma, a hipótese de que lojas com uma capacidade de estoque maior vendem mais também depende de informações sobre o estoque que não temos atualmente. Por outro lado, podemos considerar a hipótese de que lojas de maior porte tendem a ter um maior volume de vendas, uma vez que esse atributo já está presente em nosso conjunto de dados.

Seguindo para as hipóteses relacionadas aos **produtos**, podemos explorar a hipótese de que lojas que investem mais em marketing têm um maior volume de vendas. Essa hipótese se baseia na ideia de que o marketing eficaz pode atrair mais clientes e impulsionar as vendas. No entanto, é importante observar que não possuímos informações sobre investimentos em marketing específicos no momento. Além disso, podemos considerar a hipótese de que lojas que expõem mais seus produtos nas vitrines tendem a ter um maior volume de vendas. Essa hipótese se baseia na suposição de que uma exposição mais visível pode atrair a atenção dos clientes e aumentar as chances de venda.

Continuando com as hipóteses, podemos considerar a relação entre os preços dos produtos e as vendas. A hipótese de que lojas que oferecem preços mais baixos têm um maior volume de vendas reflete a preferência dos consumidores por preços acessíveis. No entanto, não possuímos informações específicas sobre os preços dos produtos no momento. Da mesma forma, a hipótese de que lojas com promoções mais agressivas e descontos maiores têm um maior volume de vendas requer dados sobre as promoções, os quais não estão disponíveis no momento. É importante ressaltar que essas hipóteses podem ser avaliadas quando os dados correspondentes estiverem disponíveis.

Por fim, passamos para as hipóteses relacionadas ao **tempo**. Podemos considerar a hipótese de que as lojas têm um maior volume de vendas durante o feriado de Natal. Essa hipótese pode ser avaliada com base nos registros dos feriados presentes em nosso conjunto de dados. Além disso, podemos explorar a hipótese de que as lojas têm um maior volume de vendas no segundo semestre do ano, após o dia 10 de cada mês, durante os finais de semana e nos períodos escolares. Essas hipóteses podem ser avaliadas utilizando informações temporais, como a data, o mês e o ano presentes em nosso conjunto de dados.

Essas são as hipóteses que podemos avaliar no momento, levando em consideração a disponibilidade dos dados. Cada uma dessas hipóteses será analisada e testada durante o processo de análise de dados, buscando compreender a correlação e a força dessas relações com o fenômeno das vendas.

## 2.3. Feature Engineering
Após finalizarmos a lista final de hipóteses, avançamos para a próxima etapa do projeto, que é a criação das variáveis. Nessa subseção, iremos derivar as variáveis necessárias para o nosso modelo de previsão de vendas.

Dentre as variáveis que iremos derivar, incluem-se o ano, o mês, o dia, a semana do ano, a semana do mês e o dia da semana. Essas variáveis temporais serão utilizadas para análises e comparações ao longo do tempo. Além disso, iremos derivar uma variável chamada  e 'promo_since' desde quando temos a promoção ativa e "competition_since" que indicará o tempo decorrido desde a ocorrência de uma competição no mercado. Essa variável será útil para avaliar como a competição impacta nas vendas. Também derivaremos uma variável relacionada aos feriados 'state_holiday', que informará quanto tempo se passou desde o último feriado, a fim de avaliar se os feriados influenciam nas vendas.

A derivação das variáveis é uma etapa simples, mas essencial, para a análise dos dados. Com base nas informações de data disponíveis, calcularemos essas variáveis para enriquecer nosso conjunto de dados e permitir análises mais precisas.






In [ ]:
df2 = df1.copy()

### Derivação de Datas
Vamos transformar a coluna "date" em formato de data utilizando o método "to_datetime". Em seguida, extrairemos o ano, mês e semana do ano usando o método "dt" e copiaremos essas informações para as colunas "year", "month" e "week_of_year", respectivamente.

Também criaremos a coluna "day" para extrair o dia da semana utilizando o método "dayofweek" e formatá-lo adequadamente usando o método "strftime".

Após executar essas operações, verificaremos os resultados usando a função "head()" para visualizar as novas colunas criadas e confirmar se o processo foi realizado corretamente. Essas novas variáveis serão úteis para análises futuras no projeto de previsão de vendas.

In [ ]:
# Year
df2['year'] = df2['date'].dt.year

# Month
df2['month'] = df2['date'].dt.month

# Day
df2['day'] = df2['date'].dt.day

# Week of Year
df2['week_of_year'] = df2['date']dt.weekofyear

# Year Week
df2['year_week'] = df2['date']dt.strtime('%Y-%W')

### Competition Since

ara calcular o tempo entre duas datas no contexto do nosso projeto de previsão de vendas, precisamos ter as duas datas disponíveis. No caso, já possuímos a data do "date". No entanto, também temos a informação de "competition", que está dividida em ano, mês e dia. Para que possamos calcular o tempo entre essas duas datas, precisamos juntar essas informações em uma única data e realizar a subtração entre elas.

Para fazer isso, utilizaremos o método "datetime" da classe correspondente. Faremos uso dos dados de ano, mês e dia da coluna "competition" e criaremos uma nova coluna chamada "competition_open_since...". Utilizaremos o método "apply" em conjunto com a função lambda para aplicar essa operação em todas as linhas da coluna.

O resultado dessa operação será uma nova data que representa a combinação do ano, mês e dia da coluna "competition_since".

In [ ]:
df2['competition_since'] = df2.apply(lambda x: datetime.datetime(
    year = x['competition_open_since_year'], month = x['competition_open_since_month'], day=1,), axis=1)

Em seguida, dividiremos esse resultado por 30, com o objetivo de manter a unidade de tempo em meses. Isso ocorre porque queremos manter a granularidade mensal nesse caso específico.

O resultado dessa operação será uma nova data que representa o tempo de competição em meses.Em seguida, dividiremos esse resultado por 30, com o objetivo de manter a unidade de tempo em meses. Isso ocorre porque queremos manter a granularidade mensal nesse caso específico.

In [ ]:
df2['competition_time_month'] = (
    (df2['date'] - df2['competition_since'])/30).apply(lambda x: x.days).astype(int)

O valor resultante será armazenado em uma nova coluna chamada "competition_time_month". Essa coluna conterá o tempo transcorrido em meses desde o início da competição. Para converter os valores para formato numérico, utilizaremos o método "astype(int)".

Esse é o procedimento para calcular o tempo em meses desde o início da competição, utilizando as informações disponíveis em nosso conjunto de dados.

### Competition Since


Primeiro, criamos uma nova coluna chamada "promo_since" no DataFrame "df2". Essa coluna é formada pela concatenação das colunas "promo2_since_year" e "promo2_since_week", convertidas para o formato de string (texto). O resultado é uma representação da data de início da promoção.

In [ ]:
df2['promo_since'] = df2['promo2_since_year'].astype(str) + '-' + df2['promo2_since_week'].astype(str)
df2['promo_since'] = df2['promo_since'].apply(lambda x: datetime.datetime.strptime(x + '-1', '%Y-%W-%w') - datetime.datetimedelta(days=7))
df2['promo_time_week'] = ((df2['date'] - df2['promo_since'])/7).apply(lambda x: x.days).astype(int)

Em seguida, utilizamos a função "apply" juntamente com a expressão lambda para aplicar uma operação em cada valor da coluna "promo_since". Essa operação é responsável por transformar a string em uma data válida. É utilizado o método "strptime" da biblioteca "datetime" para realizar essa conversão. A parte "+ '-1'" é adicionada para especificar o primeiro dia da semana correspondente à semana do ano indicada. O resultado é subtraído de um período de sete dias (datetime.timedelta(days=7)) para ajustar corretamente a data.

Por fim, criamos uma nova coluna chamada "promo_time_week" no DataFrame "df2". Essa coluna é obtida calculando a diferença em semanas entre a coluna "date" (data da venda) e a coluna "promo_since" (data de início da promoção). Primeiro, subtraímos as duas datas e dividimos o resultado por sete para obter o número de semanas. Em seguida, utilizamos a função "apply" com a expressão lambda para extrair o número de dias a partir do objeto timedelta resultante. Por fim, convertemos o resultado para o tipo inteiro (int) e armazenamos na coluna "promo_time_week".

In [ ]:
df2.head().T

### Assortment

Utilizamos a função "apply" juntamente com uma expressão lambda para aplicar uma lógica condicional em cada valor da coluna "assortment". Essa lógica condicional verifica o valor de cada elemento: se o valor for igual a 'a', atribuímos o valor 'basic'; se for igual a 'b', atribuímos o valor 'extra'; caso contrário, atribuímos o valor 'extended'.

Dessa forma, estamos mapeando os diferentes valores presentes na coluna "assortment" para categorias mais descritivas e compreensíveis: 'basic', 'extra' e 'extended'.

In [ ]:
df2['assortment'] = df2['assortment'].apply(lambda x: 'basic' if x == 'a'
                                            else 'extra' if x == 'b'
                                            else 'extended')

### State Holiday

Aqui aplicamos a mesma lógica anterior de Assortment para State Holiday:

In [ ]:
df2['state_holiday'] = df2['state_holiday'].apply(lambda x: 'public_holiday' if x == 'a'
                                            else 'easter_holiday' if x == 'b'
                                            else 'christmas' if x == 'c'
                                            else 'regular_day')

## Status do Projeto
Vamos recapitular brevemente o que já foi realizado até agora.

No primeiro passo, realizamos a descrição dos dados, resumindo as informações por meio de medidas de análise descritiva. Além disso, substituímos valores ausentes e trabalhamos com diferentes formas de dados.

Em seguida, no segundo passo, trabalhamos na identificação das características relevantes, criando o mapa mental de hipóteses. Definimos o fenômeno que queremos modelar, identificamos os agentes envolvidos e seus atributos, e elaboramos uma lista de hipóteses. Priorizamos essas hipóteses com base nos dados disponíveis no momento.

Agora, estamos adentrando o terceiro passo, que trata da filtragem de variáveis. É importante entender a diferença entre filtrar e selecionar variáveis.

A filtragem de variáveis envolve a exclusão ou retenção de variáveis com base em critérios específicos, como relevância, qualidade dos dados, correlações, entre outros. Por outro lado, a seleção de variáveis refere-se à escolha das variáveis mais importantes para a análise ou modelo preditivo, visando a redução da dimensionalidade e o aumento da eficiência computacional.

Nas etapas anteriores, realizamos o treinamento com os dados, fazendo o download dos mesmos da plataforma do Kaggle. Em seguida, passamos pela etapa de limpeza dos dados, onde fizemos a descrição dos dados e tratamos possíveis problemas de qualidade. Prosseguindo, chegamos à etapa de exploração dos dados, que realizamos no passo 2, passaremos ainda pelas etapas de Modelagem dos Dados, Algoritmos de Machine Learning, Avaliação do algoritmo e finalmente colocar o Modelo Em Produção.


# 3.0 Filtragem de Variáveis

### Filtragem
A motivação por trás da filtragem de variáveis é lidar com as restrições impostas pelo contexto de negócio. Em outras palavras, é garantir que o modelo desenvolvido possa ser implementado com sucesso e atender às necessidades da empresa.

Muitas vezes, ao iniciar um projeto de ciência de dados, todas as etapas são realizadas, mas no final, descobre-se que o modelo não pode ser colocado em produção. Isso ocorre principalmente quando as **restrições de negócio não foram consideradas desde o início do projeto**.

Uma solução para evitar essa situação é **pensar nas restrições de negócio logo no início do projeto**, antes mesmo de começar a explorar os dados. É por isso que incluímos essa etapa em nosso processo.

### Seleção
A seleção de variáveis está estritamente relacionada às **variáveis mais relevantes para o modelo**. Nesse processo, o algoritmo analisa as correlações entre as variáveis de entrada e a variável resposta, bem como as relações entre as próprias variáveis de entrada. Com base nessa análise, o algoritmo decide quais variáveis são mais relevantes para o modelo.

No entanto, é importante destacar que a **seleção de variáveis não leva em consideração as restrições do negócio**. Essa responsabilidade cabe ao cientista de dados, que deve compreender os processos e as restrições enfrentadas pelas equipes de negócio. É fundamental identificar quais restrições e problemas de dados são relevantes para o contexto específico e incorporá-los no modelo.

Portanto, a filtragem de variáveis neste projeto de previsão de vendas Rossmann, considera tanto a seleção de variáveis relevantes para o modelo quanto as restrições e problemas específicos enfrentados pelas equipes de negócio. Essa abordagem permite desenvolver um modelo mais preciso e alinhado às necessidades da empresa.

In [ ]:
df3 = df2.copy()

## 3.1 Filtragem Linhas
A coluna "Customers" indica o número de clientes que estiveram presentes na loja no dia em que as vendas foram registradas. No entanto, para o contexto do nosso projeto, não temos essa informação disponível para as próximas seis semanas, pois não sabemos quantos clientes estarão presentes nesse período. Portanto, não podemos utilizar essa coluna como variável de entrada em nossa previsão de vendas.

Outra coluna que consideramos é a coluna "Open", que indica se a loja estava aberta ou fechada no dia correspondente. Quando a loja está fechada, as vendas são registradas como zero. Nesse caso, não há aprendizado relevante, pois é esperado que as vendas sejam zero quando a loja está fechada. Por isso, optamos por excluir as linhas em que a coluna "Open" é igual a zero, indicando que a loja estava fechada. Dessa forma, estamos filtrando apenas as vendas em que as lojas estavam abertas, eliminando as vendas registradas quando a loja estava fechada.

Com essas considerações, selecionamos as colunas relevantes para o nosso modelo de previsão de vendas, excluindo as informações que não estão disponíveis no momento da predição e filtrando apenas as vendas ocorridas quando as lojas estavam abertas. Essa abordagem garante que estamos utilizando os dados disponíveis de maneira adequada e alinhada às restrições de negócio do projeto.

In [ ]:
df3 = df3[(df3['open'] != 0)  & (df3['sales'] > 0)]

Aqui estamos filtrando o DataFrame df3. Estamos selecionando apenas as linhas em que a coluna "open" é diferente de zero (ou seja, apenas as linhas em que a loja estava aberta) e a coluna "sales" é maior que zero (ou seja, apenas as linhas em que as vendas foram registradas). Dessa forma, estamos removendo as linhas em que a loja estava fechada ou as vendas foram igual a zero.






## 3.2 Seleção Colunas


Criamos uma lista chamada "cols_drop" que contém os nomes das colunas que desejamos remover, como 'customers', 'open', 'promo_interval' e 'month_map'.

Em seguida, utilizamos o método "drop" do pandas no DataFrame df3, passando como parâmetros a lista de colunas a serem excluídas (cols_drop) e o argumento "axis=1" para indicar que estamos excluindo colunas.

In [ ]:
cols_drop = ['customers','open', 'promo_interval', 'month_map']
df3 = df3.drop(cols_drop, axis=1)

df.columns